In [7]:
library(tidyverse)
library(repr)
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org



In [40]:
tennis_data <- read_csv("data/player_stats.csv")
tidy_tennis <- tennis_data |>
     select(Age:Plays,("Current Rank"):("Prize Money"))|> #Removes columns with high proportions of NA 
     separate(col=Age,into=c("age","birth_date"),sep=" ")|> #Separates age and birth date into seperate columns
     separate(col=("Current Rank"),into=c("current_rank","points"),sep=" ")|> #Separates current rank and points into separate columns)
     separate(col=("Best Rank"),into=c("best_rank", "best_rank_date"),sep=" ") #Separates best rank and date of best rank into separate columns
tidy_tennis

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,birth_date,Country,Plays,current_rank,points,best_rank,best_rank_date,Name,Backhand,Prize Money
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
26,(25-04-1993),Brazil,Right-handed,378,(97),363,(04-11-2019),Oscar Jose Gutierrez,NA,NA
18,(22-12-2001),United Kingdom,Left-handed,326,(119),316,(14-10-2019),Jack Draper,Two-handed,"$59,040"
32,(03-11-1987),Slovakia,Right-handed,178,(280),44,(14-01-2013),Lukas Lacko,Two-handed,"US$3,261,567"
21,(29-05-1998),"Korea, Republic of",Right-handed,236,(199),130,(10-04-2017),Duck Hee Lee,Two-handed,"$374,093"
27,(21-10-1992),Australia,Right-handed,183,(273),17,(11-01-2016),Bernard Tomic,Two-handed,"US$6,091,971"
22,(11-02-1997),Poland,Right-handed,31,(1398),31,(20-01-2020),Hubert Hurkacz,Two-handed,"$1,517,157"
28,(18-11-1991),United States,Right-handed,307,(131),213,(31-10-2016),Sekou Bangoura,Two-handed,"$278,709"
21,(12-05-1998),"Taiwan, Province of China",Right-handed,232,(205),229,(04-11-2019),Tung Lin Wu,Two-handed,"$59,123"
25,(29-07-1994),Uzbekistan,Right-handed,417,(81),253,(17-07-2017),Sanjar Fayziev,Two-handed,"$122,734"


In [41]:
git_add

ERROR: Error in eval(expr, envir, enclos): object 'git_add' not found
